<a href="https://colab.research.google.com/github/atharvnaidu/SentenceClassification/blob/main/SentenceIdentificationV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##import statements:
!pip install --upgrade torchtext

import torch, torchtext
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.functional import to_tensor
import os
import pandas as pd
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

/usr/local/lib/python3.10/dist-packages/torchtext/functional.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is depre

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def transformSentence(sentence, maxLength):
  tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
  tokens = tokenizer(sentence)
  vocab = build_vocab_from_iterator([tokens])
  ids = [float(vocab[token]) for token in tokens]
  padded_ids = torch.nn.functional.pad(torch.tensor(ids), (0, maxLength - len(ids)))
  return padded_ids

def transformLabel(label):
  return torch.tensor(label).long()

In [4]:
class CustomSentenceDataset(Dataset):
    def __init__(self, csv, transform=None, target_transform=None):
        self.csv = csv
        self.transform = transform
        self.target_transform = target_transform
        self.data = pd.read_csv(self.csv).iloc[:200]
        self.maxLength = 50

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 2]
        if self.transform:
            sentence = self.transform(sentence, self.maxLength)
        if self.target_transform:
            label = self.target_transform(label)
        return sentence, label

In [5]:
training_data = CustomSentenceDataset("/content/questions_vs_statements_v1.0.csv",
                                      transform=transformSentence, target_transform=transformLabel)
batch_size = 8
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
train_features, train_labels = next(iter(train_dataloader))
train_labels = train_labels.long()
print(train_labels)

tensor([1, 1, 1, 0, 1, 1, 0, 1])


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Flatten starting from dimension 1 to handle batches correctly
        self.flatten = nn.Flatten(start_dim=1)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(50, 50//2),
            nn.ReLU(),
            nn.Linear(50//2, 50//2),
            nn.ReLU(),
            nn.Linear(50//2, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(train_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.650056  [    8/  200]
Test Error: 
 Accuracy: 51.0%, Avg loss: 0.721772 

Epoch 2
-------------------------------
loss: 0.549681  [    8/  200]
Test Error: 
 Accuracy: 54.5%, Avg loss: 0.708001 

Epoch 3
-------------------------------
loss: 0.711044  [    8/  200]
Test Error: 
 Accuracy: 54.5%, Avg loss: 0.696466 

Epoch 4
-------------------------------
loss: 0.687148  [    8/  200]
Test Error: 
 Accuracy: 55.5%, Avg loss: 0.687497 

Epoch 5
-------------------------------
loss: 0.743187  [    8/  200]
Test Error: 
 Accuracy: 56.5%, Avg loss: 0.679004 

Epoch 6
-------------------------------
loss: 0.768542  [    8/  200]
Test Error: 
 Accuracy: 58.0%, Avg loss: 0.671396 

Epoch 7
-------------------------------
loss: 0.606073  [    8/  200]
Test Error: 
 Accuracy: 58.0%, Avg loss: 0.664387 

Epoch 8
-------------------------------
loss: 0.617234  [    8/  200]
Test Error: 
 Accuracy: 59.5%, Avg loss: 0.658363 

Epoch 9
----------------

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'trained_model_weights.pth')
# Alternatively, save the entire model
torch.save(model, 'trained_model.pth')

In [ ]:
model.load_state_dict(torch.load('/content/trained_model_weights.pth'))
model.eval()

In [ ]:
def predict(sentence, model):
    model.eval()
    with torch.no_grad():
        # Tokenize and transform the input sentence
        transformedSentence = transformSentence(sentence, 100)
        tensor = torch.unsqueeze(transformedSentence, 0)
        logits = model(tensor)
        pred_probab = nn.Softmax(dim=1)(logits)
        pred_label = pred_probab.argmax(1).item()
        return pred_label, pred_probab

# Example sentence
sentence = " The President of BYU, currently Kevin J Worthen, reports to the Board, through the Commissioner of Education."
pred_label, pred_probab = predict(sentence, model)
print(f"Predicted class: {pred_label}, Probabilities: {pred_probab}")